# Introduction to Code Contributions

In [ ]:
import sys
import numpy as np
sys.path.append("../")

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from aif360.algorithms.inprocessing import PrejudiceRemover

from long_term_fairness.ltf_plot import LongTermFairnessPlot
from long_term_fairness.ltf_data.group_data_generator import DataGenerator as GrpDataGen
from long_term_fairness.ltf_aif.longterm_aif import AifLongTermMetric, AifLongTermPrediction

# 1. Data Generator
Generates points given data from the previous steps.

In [ ]:
generator = IndDataGen(mean_pos=[10, 7], 
                       cov_pos=[[1.2, 0], [0, 1.3]],
                       mean_neg=[0, 1], 
                       cov_neg=[[1, 0], [0, 1.2]], 
                       degree=3, 
                       discrimination_factor=.7,
                       num_positive_label=500, 
                       num_negative_label=500, 
                       neg_label=0, 
                       pos_label=1, 
                       neg_class=0, 
                       pos_class=1)

In [ ]:
generator.sample() # must also work for None, None, None

In [ ]:
generator.get_label()

# 2. AIF360 Long Term Metric
Interface to use AIF360 metrics for long term prediction

In [ ]:
metric = AifLongTermMetric(metrics=["accuracy", "disparate_impact"],
                           pos_class=1, 
                           neg_class=0, 
                           pos_label=1, 
                           neg_label=0)

In [ ]:
metric.metric(X, 
              X_sens, 
              y, 
              y_hat)

# 3. AIF360 Long Term Prediction
Interface to use AIF360s algorithms for the long term plot

In [ ]:
clf = AifLongTermPrediction(clf=PrejudiceRemover(), 
                            pos_class=1, 
                            neg_class=0, 
                            pos_label=1, 
                            neg_label=0)

In [ ]:
clf.fit(X, 
        X_sens, 
        y)

In [ ]:
clf.predict(X, 
            X_sens)

# 4. Long Term Plot
Runs decision process over several generations.

In [ ]:
l = LongTermFairnessPlot(generator,     # must implement .sample() function
                         clf,           # must implement fit(X, X_sens, y) and predict(X, X_sense)
                         metric.metric, # function
                         x_lim=[-3, 12], 
                         y_lim=[-1, 13])